# First Experiments

## 1. Import libraries and dataset

In [ ]:
import pandas as pd
import numpy as np
import join
import sklearn

# specify the path to file
joiner = join.joiner("D:\Projects\Data Analysis\Robot Assisted surgery EDA\RCS_X_Xi_metrics_report_with_LoS.xlsx")
sheets = joiner.get_sheets()
metrics = sheets['X_Xi_metrics_report']
los_data = sheets['X_Xi_counts']
# - rename the session identifier col so tha tboth data set have the same col name.
metrics.rename(columns={'session_name': 'session_name'}, inplace=True)

## 2. Build the dataset

In [ ]:
# - data feature columns of interest
feature_col = ['session_name','proc_type','task_name','task_id','time_start','time_stop','evtcnt_camera_control','evtcnt_head_out','energy_totalduration','eom_usm1','eom_usm2','eom_usm3','eom_usm4']
metrics = metrics[feature_col]


In [ ]:
# extract dataframes for hysterectomy and lobectomy using boolean indexing
suregeries = ['hysterectomy', 'lobectomy']

# Using boolean indexing to filter the DataFrame
metrics = metrics[metrics['proc_type'].isin(suregeries)]
los_data = los_data[los_data['proc_type'].isin(suregeries)]
print(f"Surgereis in metrics :{metrics['proc_type'].unique()}")   # - confirm that only the metioned surgeries are considered
print(f"Surgeries in los sheet:{los_data['proc_type'].unique()}") # - confirm that only the metioned surgeries are considered

In [ ]:
# - get total task duration column from the start and end time.
metrics['time_start'] = pd.to_timedelta(metrics['time_start'])
metrics['time_stop'] = pd.to_timedelta(metrics['time_stop'])
metrics['task_duration'] = (metrics['time_stop'] - metrics['time_start']).dt.total_seconds()

# Rearrange column order
columns = list(metrics.columns)
columns.remove('task_duration')
columns.insert(5, 'task_duration')
columns.remove('time_start')
columns.remove('time_stop')
metrics = metrics[columns]


In [ ]:
# split the dataframes based for different surgereis
hysterectomy_metrics = metrics[metrics['proc_type'].isin(['hysterectomy'])]
hysterectomy_los = los_data[los_data['proc_type'].isin(['hysterectomy'])]
lobectomy_metrics = metrics[metrics['proc_type'].isin(['lobectomy'])]
lobectomy_los = los_data[los_data['proc_type'].isin(['lobectomy'])]

### Hysterectomy dataset

In [ ]:
# groub by session id,task name and task id
hysterectomy_metrics = hysterectomy_metrics.drop(columns=['proc_type'])
hysterectomy_metrics = hysterectomy_metrics.groupby(['session_name','task_name','task_id']).sum().reset_index()
print(f"The no. of unique surgeical cases in metrics set is :{len(hysterectomy_metrics['session_name'].unique())}")
print(f"The no. of unique tasks present in the entire set is :{len(hysterectomy_metrics['task_name'].unique())}")
print(f"The frequency tasks is as follows :{hysterectomy_metrics['task_name'].value_counts()}")

If we consider all the tasks as pivotal to the surgery, the dimensional spac ewould be very high. Given the dataset has limited no. of feature vectors(unique surgical sessions) high  dimensionality would make it even harder to derive relations. One option is to omit the taks labeled as optional. Therefore the non option tasks would be 13. But there are certain optional taks is often perfomed(high frequency of occurence.) 

In [ ]:
# Unique session IDs
session_ids = hysterectomy_metrics['session_name'].unique()

# List of hysterectomy tasks
hysterectomy_tasks = ['division of the broad ligament (right side)', 'bladder flap creation',
                      'dissection of ip ligament (right side) (optional)', 'division of the broad ligament (left side)',
                      'colpotomy', 'division of uterine vessels (right side)', 'removal of the uterus',
                      'vaginal cuff closure', 'dissection of ip ligament (left side) (optional)',
                      'division of the round ligament (left side)', 'division of the round ligament (right side)',
                      'division of uterine vessels (left side)', 'mobilize colon / removal of adhesions (optional)']

# Create a DataFrame with all combinations of session IDs and tasks
all_tasks_df = pd.DataFrame([(sid, task) for sid in session_ids for task in hysterectomy_tasks],
                            columns=['session_name', 'task_name'])

# Display the DataFrame
#print(all_tasks_df)

In [ ]:
# standardised the procedure set
merged_df = pd.merge(all_tasks_df, hysterectomy_metrics, on=['session_name', 'task_name'], how='left')
merged_df.fillna(value=np.nan, inplace=True)
merged_df = merged_df.groupby(['session_name', 'task_name']).sum().reset_index()

In [ ]:
print(merged_df.columns)

In [ ]:
# normalise using task duration and drop task duration column
columns_to_normalize = ['evtcnt_camera_control', 'evtcnt_head_out', 'energy_totalduration',
                        'eom_usm1', 'eom_usm2', 'eom_usm3', 'eom_usm4']
for col in columns_to_normalize:
    merged_df[col] = merged_df[col] / merged_df['task_duration']
merged_df = merged_df.drop(columns='task_duration')

In [ ]:
# restructure to have sinlge row for each session
def restructure(urology):
    # restructure data frame, such that each row is a single session id
    result = pd.DataFrame()
    result_temp = None
    i = 0
    counter = 0
    col_pointer = 0
    for session_id in urology['session_name']:
        if session_id == urology.iloc[col_pointer,0]:
            if i == col_pointer:
                result_temp = pd.DataFrame()
                df2 = pd.DataFrame(urology.iloc[i,:]).T
                result_temp.index = df2.index = [0]
                result_temp = pd.concat([result_temp,df2],axis=1)
            else:
                df2 = pd.DataFrame(urology.iloc[i,2:]).T
                result_temp.index = df2.index = [0]
                result_temp = pd.concat([result_temp,df2],axis=1)
            i = i +1
        else:
            result.reset_index(drop=True, inplace=True)
            result = pd.concat([result,pd.DataFrame(result_temp)],axis = 0)
            col_pointer = i
            if i == col_pointer:
                result_temp = pd.DataFrame()
                df2 = pd.DataFrame(urology.iloc[i,:]).T
                result_temp.index = df2.index = [0]
                result_temp = pd.concat([result_temp,df2],axis=1)
            else:
                df2 = pd.DataFrame(urology.iloc[i,2:]).T
                result_temp.index = df2.index = [0]
                result_temp = pd.concat([result_temp,df2],axis=1)
            i = i +1
    return result

restruct_df = restructure(merged_df)

In [ ]:
# join with los_data
hysterectomy_los = hysterectomy_los[['session_name','Length of Stay (days)']]
hysterectomy_dataset = pd.merge(restruct_df, hysterectomy_los, on=['session_name'], how='inner')

### Lobectomy Dataset

In [ ]:
print(lobectomy_metrics.columns)

In [ ]:
# groub by session id,task name and task id
lobectomy_metrics = lobectomy_metrics.drop(columns=['proc_type'])
lobectomy_metrics = lobectomy_metrics.groupby(['session_name','task_name','task_id']).sum().reset_index()
print(f"The no. of unique surgeical cases in metrics set is :{len(lobectomy_metrics['session_name'].unique())}")
print(f"The no. of unique tasks present in the entire set is :{len(lobectomy_metrics['task_name'].unique())}")
print(f"The frequency tasks is as follows :{lobectomy_metrics['task_name'].value_counts()}")


In [ ]:
# Unique session IDs
session_ids = lobectomy_metrics['session_name'].unique()

# List of hysterectomy tasks
lobectomy_tasks = [
    'Dissect Arteries (Lobe-Specific)',
    'Dissect Lymph Nodes (Excluding Subcarinal and Mediastinal)',
    'Dissect Bronchus (Lobe Specific)',
    'Dissect Veins (Lobe Specific)',
    'Divide Inferior Pulmonary Ligament',
    'Dissect Station 7 Lymph Node (Subcarinal)',
    'Mediastinal Lymph Node Dissection (Station 2,4 on R or 5, 6 on L)',
    'Hilar Dissection',
    'Fissure Dissection',
    'Divide Arteries (Lobe Specific)',
    'Divide Veins (Lobe Specific)',
    'Divide Bronchus',
    'divide arteries (lobe-specific)',
    'fissure dissection',
    'divide bronchus',
    'divide veins (lobe-specific)'
]

# Create a DataFrame with all combinations of session IDs and tasks
all_tasks_df = pd.DataFrame([(sid, task) for sid in session_ids for task in lobectomy_tasks],
                            columns=['session_name', 'task_name'])

# Display the DataFrame
#print(all_tasks_df)

In [ ]:
# standardised the procedure set
merged_df = pd.merge(all_tasks_df, lobectomy_metrics, on=['session_name', 'task_name'], how='left')
merged_df.fillna(value=np.nan, inplace=True)
merged_df = merged_df.groupby(['session_name', 'task_name']).sum().reset_index()

In [ ]:
# normalise using task duration and drop task duration column
columns_to_normalize = ['evtcnt_camera_control', 'evtcnt_head_out', 'energy_totalduration',
                        'eom_usm1', 'eom_usm2', 'eom_usm3', 'eom_usm4']
for col in columns_to_normalize:
    merged_df[col] = merged_df[col] / merged_df['task_duration']
merged_df = merged_df.drop(columns='task_duration')


In [ ]:
# restructure to have sinlge row for each session
def restructure(urology):
    # restructure data frame, such that each row is a single session id
    result = pd.DataFrame()
    result_temp = None
    i = 0
    counter = 0
    col_pointer = 0
    for session_id in urology['session_name']:
        if session_id == urology.iloc[col_pointer,0]:
            if i == col_pointer:
                result_temp = pd.DataFrame()
                df2 = pd.DataFrame(urology.iloc[i,:]).T
                result_temp.index = df2.index = [0]
                result_temp = pd.concat([result_temp,df2],axis=1)
            else:
                df2 = pd.DataFrame(urology.iloc[i,2:]).T
                result_temp.index = df2.index = [0]
                result_temp = pd.concat([result_temp,df2],axis=1)
            i = i +1
        else:
            result.reset_index(drop=True, inplace=True)
            result = pd.concat([result,pd.DataFrame(result_temp)],axis = 0)
            col_pointer = i
            if i == col_pointer:
                result_temp = pd.DataFrame()
                df2 = pd.DataFrame(urology.iloc[i,:]).T
                result_temp.index = df2.index = [0]
                result_temp = pd.concat([result_temp,df2],axis=1)
            else:
                df2 = pd.DataFrame(urology.iloc[i,2:]).T
                result_temp.index = df2.index = [0]
                result_temp = pd.concat([result_temp,df2],axis=1)
            i = i +1
    return result

restruct_df = restructure(merged_df)

In [ ]:
# join with los_data
los_data = los_data[['session_name','Length of Stay (days)']]
lobectomy_dataset = pd.merge(restruct_df, los_data, on=['session_name'], how='inner')

In [ ]:
# make sure the task_id values are identical for the same task (fill the taskid col with highest occuring value)
for col in range(len(hysterectomy_dataset.columns)):
    if hysterectomy_dataset.columns[col] == "task_id":
        value_counts = hysterectomy_dataset.iloc[:,col].value_counts()
        most_common_value = value_counts.idxmax()
        if most_common_value == 0:
            most_common_value = hysterectomy_dataset.iloc[:,col].max()
        hysterectomy_dataset.iloc[:,col] = most_common_value
for col in range(len(lobectomy_dataset.columns)):
    if lobectomy_dataset.columns[col] == "task_id":
        value_counts = lobectomy_dataset.iloc[:,col].value_counts()
        most_common_value = value_counts.idxmax()
        if most_common_value == 0.0:
            most_common_value = lobectomy_dataset.iloc[:,col].max()
        lobectomy_dataset.iloc[:,col] = most_common_value

In [ ]:
task_relset = hysterectomy_dataset.drop(columns = ['task_name','evtcnt_camera_control', 'evtcnt_head_out', 'energy_totalduration',
                        'eom_usm1', 'eom_usm2', 'eom_usm3', 'eom_usm4'])
col_h = task_relset.columns.tolist()
for i in range(len(col_h)):
    if col_h[i] == "task_id":
        value_counts = task_relset.iloc[:,i].value_counts()
        most_common_value = value_counts.idxmax()
        if most_common_value == 0:
            most_common_value = task_relset.iloc[:,col].max()
        col_h[i] = f"{most_common_value}" + "task_id"
task_relset.columns = col_h
def replace_nonzero_with_one_and_zero_with_zero(x):
    if x != 0:
        return 1
    else:
        return 0
selected_columns = [col for col in task_relset.columns if col != 'Length of Stay (days)']
los = task_relset['Length of Stay (days)']
task_relset = task_relset[selected_columns].map(replace_nonzero_with_one_and_zero_with_zero)
task_relset = pd.concat([task_relset,los],axis=1)
print(task_relset)

In [ ]:
# make sure each column in dataframe is uniquely named
col_h = hysterectomy_dataset.columns.tolist()
for i in range(len(col_h)):
    if col_h[i] == "task_id":
        value_counts = hysterectomy_dataset.iloc[:,i].value_counts()
        most_common_value = value_counts.idxmax()
        col_h[i] = f"{most_common_value}" + "task_id"
        col_h[i+1] = f"{most_common_value}" + "evtcnt_camera_control"
        col_h[i+2] = f"{most_common_value}" + "evtcnt_head_out"
        col_h[i+3] = f"{most_common_value}" + "energy_totalduration"
        col_h[i+4] = f"{most_common_value}" + "eom_usm1"
        col_h[i+5] = f"{most_common_value}" + "eom_usm2"
        col_h[i+6] = f"{most_common_value}" + "eom_usm3"
        col_h[i+7] = f"{most_common_value}" + "eom_usm4"
hysterectomy_dataset.columns = col_h
col_l = lobectomy_dataset.columns.tolist()
for i in range(len(col_l)):
    if col_l[i] == "task_id":
        value_counts = lobectomy_dataset.iloc[:,i].value_counts()
        most_common_value = value_counts.idxmax()
        col_l[i] = f"{most_common_value}" + "task_id"
        col_l[i+1] = f"{most_common_value}" + "evtcnt_camera_control"
        col_l[i+2] = f"{most_common_value}" + "evtcnt_head_out"
        col_l[i+3] = f"{most_common_value}" + "energy_totalduration"
        col_l[i+4] = f"{most_common_value}" + "eom_usm1"
        col_l[i+5] = f"{most_common_value}" + "eom_usm2"
        col_l[i+6] = f"{most_common_value}" + "eom_usm3"
        col_l[i+7] = f"{most_common_value}" + "eom_usm4"
lobectomy_dataset.columns = col_l


In [ ]:
"""excel_writer = pd.ExcelWriter('output.xlsx', engine='xlsxwriter')
hysterectomy_dataset.to_excel(excel_writer, sheet_name='hysterectomy_dataset', index=False)
lobectomy_dataset.to_excel(excel_writer, sheet_name='lobectomy_dataset', index=False)
# Save the Excel file
excel_writer.close()"""


## Modeling

In [ ]:
import re

# define datasets
print(hysterectomy_dataset.info())
lin_hysterectomy = hysterectomy_dataset
print(lobectomy_dataset.info())
lin_lobectomy = lobectomy_dataset
# convert the columns to integers and strings
strings = [r'session_name',r'task_name',r'task_id']
floats = [r'evtcnt_camera_control',r'evtcnt_head_out',r'energy_totalduration',r'eom_usm1',r'eom_usm2',r'eom_usm3',r'eom_usm4']

#convert dtypes
for column in hysterectomy_dataset.columns:
    for patterns in strings:
        if re.search(patterns, column):
            hysterectomy_dataset[column] = hysterectomy_dataset[column].astype('string')
    for patternf in floats:
        if re.search(patternf, column):
            hysterectomy_dataset[column] = pd.to_numeric(hysterectomy_dataset[column], errors='coerce').tolist()
for column in hysterectomy_dataset.columns:
    if  re.search(r'task_id', column):
        lin_hysterectomy = lin_hysterectomy.drop(columns=column)
for column in lobectomy_dataset.columns:
    if  re.search(r'task_id', column):
        lin_lobectomy = lin_lobectomy.drop(columns=column)


### 1. Linear Regression

#### 1.1 Hysterectomy

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import numpy as np

# Define input and output variables
X = task_relset.drop(columns=['Length of Stay (days)', 'session_name'])
y = task_relset['Length of Stay (days)']


# Initialize linear regression model
model = LinearRegression()

# Perform 5-fold cross-validation
cv_scores = cross_val_score(model, X, y, cv=5, scoring='neg_mean_squared_error')

# Convert scores to positive values
cv_scores = -cv_scores

# Print cross-validation scores
print("Cross-validation scores:", cv_scores)

# Calculate mean and standard deviation of cross-validation scores
mean_cv_score = np.mean(cv_scores)
std_cv_score = np.std(cv_scores)
print("Mean CV Score:", mean_cv_score)
print("Standard Deviation of CV Scores:", std_cv_score)


### 2. Support Vector Machines

#### 2.1 Hysterectomy

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error


# Define input and output variables
X = task_relset.drop(columns=['Length of Stay (days)', 'session_name'])
y = task_relset['Length of Stay (days)']

# Replace missing values with zeros
X.fillna(0, inplace=True)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize SVM regressor
svm_regressor = SVR(kernel='linear')

# Train the SVM regressor
svm_regressor.fit(X_train, y_train)

# Predict on the testing set
y_pred = svm_regressor.predict(X_test)
# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

### 3. Decision Trees

#### 3.1 Hysterectomy

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

# Define input and output variables
X = task_relset.drop(columns=['Length of Stay (days)', 'session_name'])
y = task_relset['Length of Stay (days)']
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize Decision Tree regressor
tree_regressor = DecisionTreeRegressor()

# Train the Decision Tree regressor
tree_regressor.fit(X_train, y_train)

# Predict on the testing set
y_pred = tree_regressor.predict(X_test)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

### 4. Random Forest

#### 4.1 Hysterectomy

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

X = lin_hysterectomy.drop(columns=['Length of Stay (days)', 'task_name', 'session_name'])
y = lin_hysterectomy['Length of Stay (days)']

# Replace missing values with zeros
X.fillna(0, inplace=True)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize Random Forest regressor
forest_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the Random Forest regressor
forest_regressor.fit(X_train, y_train)

# Predict on the testing set
y_pred = forest_regressor.predict(X_test)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)

 ### 5. Neural Network

#### 5.1 Hysterectomy

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error

# Define input and output variables
X = task_relset.drop(columns=['Length of Stay (days)', 'session_name'])
y = task_relset['Length of Stay (days)']

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize MLPRegressor with a single hidden layer
mlp_regressor = MLPRegressor(hidden_layer_sizes=(13,100,1), activation='relu', solver='adam', max_iter=100, random_state=42)

# Train the MLPRegressor
mlp_regressor.fit(X_train, y_train)

# Predict on the testing set
y_pred = mlp_regressor.predict(X_test)
print(y_pred)
print(y_test)

# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error:", mse)


# Second Experiments

## Import datasets and libraries

In [28]:
import pandas as pd
import numpy as np
import join
import sklearn
import re

# specify the path to file
joiner = join.joiner("D:\Projects\Data Analysis\Robot Assisted surgery EDA\RCS_X_Xi_metrics_report_with_LoS.xlsx")
sheets = joiner.get_sheets()
metrics = sheets['X_Xi_metrics_report']
los_data = sheets['X_Xi_counts']

Finished processing and sheets returned.


In [29]:
# - data feature columns of interest
feature_col = ['session_name','proc_type','task_name','task_id','time_start','time_stop','evtcnt_camera_control','evtcnt_head_out','energy_totalduration','eom_usm1','eom_usm2','eom_usm3','eom_usm4']
metrics = metrics[feature_col]

# extract dataframes for hysterectomy and lobectomy using boolean indexing
suregeries = ['hysterectomy', 'lobectomy']

# Using boolean indexing to filter the DataFrame
metrics = metrics[metrics['proc_type'].isin(suregeries)]
los_data = los_data[los_data['proc_type'].isin(suregeries)]
print(f"Surgereis in metrics :{metrics['proc_type'].unique()}")   # - confirm that only the metioned surgeries are considered
print(f"Surgeries in los sheet:{los_data['proc_type'].unique()}") # - confirm that only the metioned surgeries are considered

# - get total task duration column from the start and end time.
metrics['time_start'] = pd.to_timedelta(metrics['time_start'])
metrics['time_stop'] = pd.to_timedelta(metrics['time_stop'])
metrics['task_duration'] = (metrics['time_stop'] - metrics['time_start']).dt.total_seconds()

# Rearrange column order
columns = list(metrics.columns)
columns.remove('task_duration')
columns.insert(5, 'task_duration')
columns.remove('time_start')
columns.remove('time_stop')
metrics = metrics[columns]

# split the dataframes based for different surgereis
hysterectomy_metrics = metrics[metrics['proc_type'].isin(['hysterectomy'])]
hysterectomy_los = los_data[los_data['proc_type'].isin(['hysterectomy'])]
lobectomy_metrics = metrics[metrics['proc_type'].isin(['lobectomy'])]
lobectomy_los = los_data[los_data['proc_type'].isin(['lobectomy'])]

Surgereis in metrics :['hysterectomy' 'lobectomy']
Surgeries in los sheet:['hysterectomy' 'lobectomy']


## 1. Averaging task and normalising using task duration for LOS prediction

### 1.1 create dataset

In [30]:
# groupy by session name and task id
hysterectomy_metrics = hysterectomy_metrics.groupby(['session_name','task_id']).sum().reset_index()

In [31]:
# average by task duration
col_to_average = ['evtcnt_camera_control','evtcnt_head_out','energy_totalduration','eom_usm1','eom_usm2','eom_usm3','eom_usm4']
for col in col_to_average:
    hysterectomy_metrics[col] = hysterectomy_metrics[col]/hysterectomy_metrics['task_duration']
# drop the task_name column
hysterectomy_metrics = hysterectomy_metrics.drop(columns = ['proc_type','task_name','task_id'])

In [32]:
# sum up to get the session features
hysterectomy_metrics = hysterectomy_metrics.groupby(['session_name']).sum().reset_index()

In [33]:
# join with los data
hysterectomy_los = hysterectomy_los[['session_name','Length of Stay (days)']]
hysterectomy_dataset = pd.merge(hysterectomy_metrics,hysterectomy_los, on ='session_name',how ='inner')

### 1.2 preprocess for ML model training and testing

In [34]:
# convert the columns to integers and strings
strings = [r'session_name']
floats = [r'evtcnt_camera_control',r'evtcnt_head_out',r'energy_totalduration',r'eom_usm1',r'eom_usm2',r'eom_usm3',r'eom_usm4']

#convert dtypes
for column in hysterectomy_dataset.columns:
    for patterns in strings:
        if re.search(patterns, column):
            hysterectomy_dataset[column] = hysterectomy_dataset[column].astype('string')
    for patternf in floats:
        if re.search(patternf, column):
            hysterectomy_dataset[column] = pd.to_numeric(hysterectomy_dataset[column], errors='coerce').tolist()

hysterectomy_dataset.to_excel('exp2_session_los.xlsx', index=False)

In [35]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import KFold, cross_val_score,train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor 
from sklearn.neural_network import MLPRegressor

# Define input and output variables
X = hysterectomy_dataset.drop(columns=['Length of Stay (days)', 'session_name'])
X = X.values
y = hysterectomy_dataset['Length of Stay (days)']
y = y.values

### 1.3 ML models

#### 1.3.1 Linear Regression

In [36]:
# Initialize the KFold object with 5 splits
kf = KFold(n_splits=5, shuffle=True, random_state=42)

mse_scores = []

# Iterate over each fold
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    # Fit the model
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Predict on the test set
    y_pred = model.predict(X_test)
    
    # Calculate mean squared error
    mse = mean_squared_error(y_test, y_pred)
    mse_scores.append(mse)
    print("Mean Squared Error for current fold:", mse)

# Calculate the average mean squared error
average_mse = np.mean(mse_scores)
print("Average Mean Squared Error:", average_mse)

Mean Squared Error for current fold: 10.974788254040636
Mean Squared Error for current fold: 1.3299471253283872
Mean Squared Error for current fold: 11.377181076389226
Mean Squared Error for current fold: 298.9213942808207
Mean Squared Error for current fold: 13.114063042460067
Average Mean Squared Error: 67.14347475580782


We can see that a single fold has a very high mse, which in turn is increasing the mean MSE. This is same for every case even if increase the no. of folds. A sinlge fold has extremely large value. 

#### 1.3.2 Decision Trees

In [37]:
kf = KFold(n_splits=3, shuffle=True, random_state=42)

mse_scores = []

for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    dt_regressor = DecisionTreeRegressor(random_state=42)
    dt_regressor.fit(X_train, y_train)
    y_pred = dt_regressor.predict(X_test)
    
    mse = mean_squared_error(y_test, y_pred)
    mse_scores.append(mse)
    print("Mean Squared Error for current fold:", mse)
    
average_mse = np.mean(mse_scores)
print("Average Mean Squared Error:", average_mse)

Mean Squared Error for current fold: 5.833333333333333
Mean Squared Error for current fold: 8.363636363636363
Mean Squared Error for current fold: 10.909090909090908
Average Mean Squared Error: 8.368686868686867


#### 1.3.3 Support Vector Machines

In [39]:
svm_df = hysterectomy_dataset
svm_df.fillna(0,inplace=True)
svm_df.reset_index(drop=True, inplace=True)
X_svm = hysterectomy_dataset.drop(columns=['Length of Stay (days)', 'session_name'])
X_svm = X_svm.values
y_svm = hysterectomy_dataset['Length of Stay (days)']
y_svm = y_svm.values

svm_regressor = SVR(kernel='linear')
kf = KFold(n_splits=3, shuffle=True, random_state=42)

mse_scores = []

for train_index, test_index in kf.split(X_svm):
    X_train, X_test = X_svm[train_index], X_svm[test_index]
    y_train, y_test = y_svm[train_index], y_svm[test_index]
    
    svm_regressor.fit(X_train, y_train)
    y_pred = svm_regressor.predict(X_test)
    
   
    mse = mean_squared_error(y_test, y_pred)
    mse_scores.append(mse)


average_mse = np.mean(mse_scores)

print("Mean Squared Error for each fold:", mse_scores)
print("Average Mean Squared Error:", average_mse)

Mean Squared Error for each fold: [40.33559629758877, 11.079061452130519, 64.83625513037157]
Average Mean Squared Error: 38.75030429336362


#### 1.3.4 Random Forest 

In [40]:
# Initialize the Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)

# Initialize the KFold object with 5 splits
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Perform 5-fold cross-validation
mse_scores = cross_val_score(rf_regressor, X, y, cv=kf, scoring='neg_mean_squared_error')

# Calculate the mean squared error scores
mse_scores = -mse_scores  # convert negative MSE scores to positive
mean_mse = np.mean(mse_scores)

print("Mean Squared Error for each fold:", mse_scores)
print("Average Mean Squared Error:", mean_mse)

Mean Squared Error for each fold: [ 7.01563651  0.89605859  2.84847817 17.44275802 11.45231343]
Average Mean Squared Error: 7.931048943915345


#### 1.3.5 Neural Networks

In [41]:
# Standardize features by removing the mean and scaling to unit variance
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

mlp_regressor = MLPRegressor(hidden_layer_sizes=(7,35,1), activation='relu', solver='adam', max_iter=5000, random_state=42)

kf = KFold(n_splits=3, shuffle=True, random_state=42)
mse_scores = cross_val_score(mlp_regressor, X_scaled, y, cv=kf, scoring='neg_mean_squared_error')

mse_scores = -mse_scores  
mean_mse = np.mean(mse_scores)

print("Mean Squared Error for each fold:", mse_scores)
print("Average Mean Squared Error:", mean_mse)

Mean Squared Error for each fold: [1.64067363 8.93619445 7.00562146]
Average Mean Squared Error: 5.860829842980972


## 2. Reducing the no. of tasks considered for LOS prediction

### preprocessing

In [42]:
import pandas as pd
import numpy as np
import join
import sklearn

# specify the path to file
joiner = join.joiner("D:\Projects\Data Analysis\Robot Assisted surgery EDA\RCS_X_Xi_metrics_report_with_LoS.xlsx")
sheets = joiner.get_sheets()
metrics = sheets['X_Xi_metrics_report']
los_data = sheets['X_Xi_counts']
# - rename the session identifier col so tha tboth data set have the same col name.
metrics.rename(columns={'session_name': 'session_name'}, inplace=True)

# - data feature columns of interest
feature_col = ['session_name','proc_type','task_name','task_id','time_start','time_stop','evtcnt_camera_control','evtcnt_head_out','energy_totalduration','eom_usm1','eom_usm2','eom_usm3','eom_usm4']
metrics = metrics[feature_col]

# extract dataframes for hysterectomy and lobectomy using boolean indexing
suregeries = ['hysterectomy', 'lobectomy']

# Using boolean indexing to filter the DataFrame
metrics = metrics[metrics['proc_type'].isin(suregeries)]
los_data = los_data[los_data['proc_type'].isin(suregeries)]
print(f"Surgereis in metrics :{metrics['proc_type'].unique()}")   # - confirm that only the metioned surgeries are considered
print(f"Surgeries in los sheet:{los_data['proc_type'].unique()}") # - confirm that only the metioned surgeries are considered

# - get total task duration column from the start and end time.
metrics['time_start'] = pd.to_timedelta(metrics['time_start'])
metrics['time_stop'] = pd.to_timedelta(metrics['time_stop'])
metrics['task_duration'] = (metrics['time_stop'] - metrics['time_start']).dt.total_seconds()

# Rearrange column order
columns = list(metrics.columns)
columns.remove('task_duration')
columns.insert(5, 'task_duration')
columns.remove('time_start')
columns.remove('time_stop')
metrics = metrics[columns]

# split the dataframes based for different surgereis
hysterectomy_metrics = metrics[metrics['proc_type'].isin(['hysterectomy'])]
hysterectomy_los = los_data[los_data['proc_type'].isin(['hysterectomy'])]
lobectomy_metrics = metrics[metrics['proc_type'].isin(['lobectomy'])]
lobectomy_los = los_data[los_data['proc_type'].isin(['lobectomy'])]

# groub by session id,task name and task id
hysterectomy_metrics = hysterectomy_metrics.drop(columns=['proc_type'])
hysterectomy_metrics = hysterectomy_metrics.groupby(['session_name','task_name','task_id']).sum().reset_index()
print(f"The no. of unique surgeical cases in metrics set is :{len(hysterectomy_metrics['session_name'].unique())}")
print(f"The no. of unique tasks present in the entire set is :{len(hysterectomy_metrics['task_name'].unique())}")
#print(f"The frequency tasks is as follows :{hysterectomy_metrics['task_name'].value_counts()}")

# Unique session IDs
session_ids = hysterectomy_metrics['session_name'].unique()

# List of hysterectomy tasks
hysterectomy_tasks = ['division of the broad ligament (right side)', 'bladder flap creation',
                      'dissection of ip ligament (right side) (optional)', 'division of the broad ligament (left side)',
                      'colpotomy', 'division of uterine vessels (right side)', 'removal of the uterus',
                      'vaginal cuff closure']

# Create a DataFrame with all combinations of session IDs and tasks
all_tasks_df = pd.DataFrame([(sid, task) for sid in session_ids for task in hysterectomy_tasks],
                            columns=['session_name', 'task_name'])

# Display the DataFrame
#print(all_tasks_df)

# standardised the procedure set
merged_df = pd.merge(all_tasks_df, hysterectomy_metrics, on=['session_name', 'task_name'], how='left')
merged_df.fillna(value=np.nan, inplace=True)
merged_df = merged_df.groupby(['session_name', 'task_name']).sum().reset_index()


# normalise using task duration and drop task duration column
columns_to_normalize = ['evtcnt_camera_control', 'evtcnt_head_out', 'energy_totalduration',
                        'eom_usm1', 'eom_usm2', 'eom_usm3', 'eom_usm4']
for col in columns_to_normalize:
    merged_df[col] = merged_df[col] / merged_df['task_duration']
merged_df = merged_df.drop(columns='task_duration')

# restructure to have sinlge row for each session
def restructure(urology):
    # restructure data frame, such that each row is a single session id
    result = pd.DataFrame()
    result_temp = None
    i = 0
    counter = 0
    col_pointer = 0
    for session_id in urology['session_name']:
        if session_id == urology.iloc[col_pointer,0]:
            if i == col_pointer:
                result_temp = pd.DataFrame()
                df2 = pd.DataFrame(urology.iloc[i,:]).T
                result_temp.index = df2.index = [0]
                result_temp = pd.concat([result_temp,df2],axis=1)
            else:
                df2 = pd.DataFrame(urology.iloc[i,2:]).T
                result_temp.index = df2.index = [0]
                result_temp = pd.concat([result_temp,df2],axis=1)
            i = i +1
        else:
            result.reset_index(drop=True, inplace=True)
            result = pd.concat([result,pd.DataFrame(result_temp)],axis = 0)
            col_pointer = i
            if i == col_pointer:
                result_temp = pd.DataFrame()
                df2 = pd.DataFrame(urology.iloc[i,:]).T
                result_temp.index = df2.index = [0]
                result_temp = pd.concat([result_temp,df2],axis=1)
            else:
                df2 = pd.DataFrame(urology.iloc[i,2:]).T
                result_temp.index = df2.index = [0]
                result_temp = pd.concat([result_temp,df2],axis=1)
            i = i +1
    return result

restruct_df = restructure(merged_df)

# join with los_data
hysterectomy_los = hysterectomy_los[['session_name','Length of Stay (days)']]
hysterectomy_dataset = pd.merge(restruct_df, hysterectomy_los, on=['session_name'], how='inner')

Finished processing and sheets returned.
Surgereis in metrics :['hysterectomy' 'lobectomy']
Surgeries in los sheet:['hysterectomy' 'lobectomy']
The no. of unique surgeical cases in metrics set is :31
The no. of unique tasks present in the entire set is :25


In [43]:
# make sure the task_id values are identical for the same task (fill the taskid col with highest occuring value)
for col in range(len(hysterectomy_dataset.columns)):
    if hysterectomy_dataset.columns[col] == "task_id":
        value_counts = hysterectomy_dataset.iloc[:,col].value_counts()
        most_common_value = value_counts.idxmax()
        if most_common_value == 0:
            most_common_value = hysterectomy_dataset.iloc[:,col].max()
        hysterectomy_dataset.iloc[:,col] = most_common_value

# make sure each column in dataframe is uniquely named
col_h = hysterectomy_dataset.columns.tolist()
for i in range(len(col_h)):
    if col_h[i] == "task_id":
        value_counts = hysterectomy_dataset.iloc[:,i].value_counts()
        most_common_value = value_counts.idxmax()
        col_h[i] = f"{most_common_value}" + "task_id"
        col_h[i+1] = f"{most_common_value}" + "evtcnt_camera_control"
        col_h[i+2] = f"{most_common_value}" + "evtcnt_head_out"
        col_h[i+3] = f"{most_common_value}" + "energy_totalduration"
        col_h[i+4] = f"{most_common_value}" + "eom_usm1"
        col_h[i+5] = f"{most_common_value}" + "eom_usm2"
        col_h[i+6] = f"{most_common_value}" + "eom_usm3"
        col_h[i+7] = f"{most_common_value}" + "eom_usm4"
hysterectomy_dataset.columns = col_h

In [44]:
import re

# define datasets
lin_hysterectomy = hysterectomy_dataset

# convert the columns to integers and strings
strings = [r'session_name',r'task_name',r'task_id']
floats = [r'evtcnt_camera_control',r'evtcnt_head_out',r'energy_totalduration',r'eom_usm1',r'eom_usm2',r'eom_usm3',r'eom_usm4']

#convert dtypes
for column in hysterectomy_dataset.columns:
    for patterns in strings:
        if re.search(patterns, column):
            hysterectomy_dataset[column] = hysterectomy_dataset[column].astype('string')
    for patternf in floats:
        if re.search(patternf, column):
            hysterectomy_dataset[column] = pd.to_numeric(hysterectomy_dataset[column], errors='coerce').tolist()
for column in hysterectomy_dataset.columns:
    if  re.search(r'task_id', column):
        lin_hysterectomy = lin_hysterectomy.drop(columns=column)
lin_hysterectomy = lin_hysterectomy.drop(columns=['task_name'])
lin_hysterectomy.to_excel('exp2_limited_feat.xlsx', index=False)

### Modeling

In [45]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor

In [46]:
X = lin_hysterectomy.drop(columns=['Length of Stay (days)', 'session_name'])
X.fillna(0,inplace=True)
y = lin_hysterectomy['Length of Stay (days)']
print(X.columns)

Index(['417.0evtcnt_camera_control', '417.0evtcnt_head_out',
       '417.0energy_totalduration', '417.0eom_usm1', '417.0eom_usm2',
       '417.0eom_usm3', '417.0eom_usm4', '422.0evtcnt_camera_control',
       '422.0evtcnt_head_out', '422.0energy_totalduration', '422.0eom_usm1',
       '422.0eom_usm2', '422.0eom_usm3', '422.0eom_usm4',
       '406.0evtcnt_camera_control', '406.0evtcnt_head_out',
       '406.0energy_totalduration', '406.0eom_usm1', '406.0eom_usm2',
       '406.0eom_usm3', '406.0eom_usm4', '416.0evtcnt_camera_control',
       '416.0evtcnt_head_out', '416.0energy_totalduration', '416.0eom_usm1',
       '416.0eom_usm2', '416.0eom_usm3', '416.0eom_usm4',
       '410.0evtcnt_camera_control', '410.0evtcnt_head_out',
       '410.0energy_totalduration', '410.0eom_usm1', '410.0eom_usm2',
       '410.0eom_usm3', '410.0eom_usm4', '561.0evtcnt_camera_control',
       '561.0evtcnt_head_out', '561.0energy_totalduration', '561.0eom_usm1',
       '561.0eom_usm2', '561.0eom_usm3', '561.0

#### 1. Linear Regression

In [47]:
model_lr = LinearRegression()
cv_scores_lr = cross_val_score(model_lr, X, y, cv=5, scoring='neg_mean_squared_error')
cv_scores_lr = -cv_scores_lr
print("Cross-validation scores:", cv_scores_lr)

# Calculate mean and standard deviation of cross-validation scores
mean_cv_score_lr = np.mean(cv_scores_lr)
std_cv_score_lr = np.std(cv_scores_lr)
print("Mean CV Score:", mean_cv_score_lr)
print("Standard Deviation of CV Scores:", std_cv_score_lr)

Cross-validation scores: [48.15771708 10.07900698  7.9465161  10.10057473  7.03121644]
Mean CV Score: 16.663006266852527
Standard Deviation of CV Scores: 15.792909420400349


#### 2. SVM

In [48]:
model_svm = SVR(kernel='linear')
cv_scores_svm = cross_val_score(model_svm, X, y, cv=3, scoring='neg_mean_squared_error')
cv_scores_svm = -cv_scores_svm
print("Cross-validation scores:", cv_scores_svm)

# Calculate mean and standard deviation of cross-validation scores
mean_cv_score_svm = np.mean(cv_scores_svm)
std_cv_score_svm = np.std(cv_scores_svm)
print("Mean CV Score:", mean_cv_score_svm)
print("Standard Deviation of CV Scores:", std_cv_score_svm)

Cross-validation scores: [ 4.96708256  0.95534679 14.03764384]
Mean CV Score: 6.653357730964918
Standard Deviation of CV Scores: 5.4723101128708835


#### decision trees

In [49]:
model_dt = DecisionTreeRegressor()
cv_scores_dt = cross_val_score(model_dt, X, y, cv=5, scoring='neg_mean_squared_error')
cv_scores_dt = -cv_scores_dt
print("Cross-validation scores:", cv_scores_dt)

# Calculate mean and standard deviation of cross-validation scores
mean_cv_score_dt = np.mean(cv_scores_dt)
std_cv_score_dt = np.std(cv_scores_dt)
print("Mean CV Score:", mean_cv_score_dt)
print("Standard Deviation of CV Scores:", std_cv_score_dt)

Cross-validation scores: [11.17857143  7.71428571  0.71428571  5.875      18.33333333]
Mean CV Score: 8.763095238095238
Standard Deviation of CV Scores: 5.859153739208636


#### 4. Random forest

In [50]:
model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
cv_scores_rf = cross_val_score(model_rf, X, y, cv=5, scoring='neg_mean_squared_error')
cv_scores_rf = -cv_scores_rf
print("Cross-validation scores:", cv_scores_rf)

# Calculate mean and standard deviation of cross-validation scores
mean_cv_score_rf = np.mean(cv_scores_rf)
std_cv_score_rf = np.std(cv_scores_rf)
print("Mean CV Score:", mean_cv_score_rf)
print("Standard Deviation of CV Scores:", std_cv_score_rf)

Cross-validation scores: [13.06657254  7.15328571  1.24536639  9.82584843 19.33751123]
Mean CV Score: 10.125716860052911
Standard Deviation of CV Scores: 6.021489517829252


#### 5. Neural network

In [51]:
model_nn = MLPRegressor(hidden_layer_sizes=(98,), activation='relu', solver='adam', max_iter=5000, random_state=42)
cv_scores_nn = cross_val_score(model_nn, X, y, cv=2, scoring='neg_mean_squared_error')
cv_scores_nn = -cv_scores_nn

print("Cross-validation scores:", cv_scores_nn)
mean_cv_score_nn = np.mean(cv_scores_nn)
std_cv_score_nn = np.std(cv_scores_nn)

print("Mean CV Score:", mean_cv_score_nn)
print("Standard Deviation of CV Scores:", std_cv_score_nn)

Cross-validation scores: [41.96199277 12.75227089]
Mean CV Score: 27.357131830896023
Standard Deviation of CV Scores: 14.604860938363451
